In [1]:
import pandas as pd
from tensorflow import keras as ks

In [62]:
train = pd.read_csv('data/split_data/train_stats.csv', index_col=0)
test = pd.read_csv('data/split_data/test_stats.csv', index_col=0)
train.head()

,tension_index_int,mode_int,std_int,mean_int,var_int,pnn50_int,RMSSD_int,ivr_int,vpr_int,papr_int,...,r-r-1,r-r-2,r-r-3,r-r-4,r-r-5,y1,y2,y3,y4,y5
0,626.566416,798.0,14.805404,799.2,1.852528,0.0,24.413111,0.010000,0.000031,0.000501,...,800,780,792,820,804,0,0,0,0,0
1,507.882334,820.4,19.677398,804.8,2.445005,0.0,27.856777,0.008333,0.000025,0.000488,...,780,792,820,804,828,0,0,0,0,0
2,203.136426,794.0,48.989795,832.0,5.888196,50.0,74.108029,0.003226,0.000010,0.000504,...,792,820,804,828,916,0,0,0,0,0
3,238.299495,807.0,68.445599,865.6,7.907301,75.0,116.584733,0.003846,0.000008,0.000743,...,820,804,828,916,960,0,0,0,0,0
4,159.063434,806.0,72.685624,892.8,8.141311,75.0,109.489726,0.002564,0.000008,0.000496,...,804,828,916,960,956,0,0,0,0,0


In [90]:
model = ks.Sequential([
    ks.layers.InputLayer(input_shape=train.iloc[:, :-5].shape[1]),
    ks.layers.LayerNormalization(),
    ks.layers.Dense(units=64, activation='tanh'),
    ks.layers.LayerNormalization(),
    ks.layers.Dense(units=128, activation='tanh'),
    ks.layers.LayerNormalization(),
    ks.layers.Dense(units=256, activation='tanh'),
    ks.layers.Dense(units=5, activation='sigmoid'),
])

In [89]:
model.compile(optimizer='adam', loss='mse', metrics='accuracy')

In [87]:
model.fit(x=train.iloc[:, :-5].values, y=train.iloc[:, -5:].values, batch_size=32, epochs=15)

Epoch 1/15
115/115 [==============================] - 2s 3ms/step - loss: 0.1029 - accuracy: 0.1992  
Epoch 2/15
115/115 [==============================] - 0s 3ms/step - loss: 0.0960 - accuracy: 0.2446

In [79]:
pred = model.predict(test.iloc[:, :-5].values)

In [80]:
df_pred = pd.DataFrame(data=pred, columns=test.iloc[:, -5:].columns)

In [84]:
ks.metrics.mse(test.iloc[:, -5:], df_pred)

<tf.Tensor: shape=(1626,), dtype=float32, numpy=
array([0.00053081, 0.00126083, 0.00289952, ..., 0.00072543, 0.00059236,
       0.00372962], dtype=float32)>

In [75]:
df_pred.loc[df_pred.y1 > 0.5]

,y1,y2,y3,y4,y5


In [71]:
test.loc[[514]]

,tension_index_int,mode_int,std_int,mean_int,var_int,pnn50_int,RMSSD_int,ivr_int,vpr_int,papr_int,...,r-r-1,r-r-2,r-r-3,r-r-4,r-r-5,y1,y2,y3,y4,y5
514,11961.722488,627.0,2.19089,625.6,0.350206,0.0,3.464102,0.15,0.000399,0.000957,...,624,624,628,628,624,0,0,0,0,0
